In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
import pandas as pd
import string
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

from afinn import Afinn
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem.lancaster import LancasterStemmer
from PIL import Image
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#lettura del csv
reviews = pd.read_csv("reviews_clean.csv")

#settare questo campo nel caso in cui il testo dei commenti sia molto lungo
pd.set_option('display.max_colwidth', -1)

notes = list(reviews.body.values)

In [0]:
import re
WORD = re.compile(r'\w+')
def regTokenize(text):
    ab = re.sub('[^A-Za-z0-9]+', ' ', text)
    words = WORD.findall(ab)
    return words

In [0]:
i = 0
for x in reviews["body"][0:10]:
    token = regTokenize(x)
    reviews["body"][i] = token
    i = i+1
print(reviews["body"][0:10])

C:\Users\marti\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0    [Non, ho, mai, scritto, una, recensione, in, vita, mia, ma, per, questo, gioco, sento, proprio, il, bisogno, di, scriverci, qualcosa, Parto, consigliando, a, tutti, di, non, acquistare, questo, gioco, perch, BUTTATE, i, vostri, i, soldi, Sono, arrivato, a, questa, conclusione, semplicemente, perch, che, tu, sia, bravo, o, meno, non, importa, e, il, gioco, che, decide, per, te, tra, 8, 9, gol, a, partita, o, errori, assurdi, fatti, dalla, cpu, il, tuo, giocare, passa, in, secondo, piano, Ormai, credo, che, la, EA, preferisca, arricchirsi, con, vendite, di, giochi, e, fifa, points, senza, pensare, minimamente, al, gamplay, o, a, rendere, il, prodotto, ...]                                             
1    [Uno, dei, peggiori, se, non, il, peggior, gioco, Fifa, degli, ultimi, anni, dovrebbe, essere, il, gioco, di, calcio, pi, realistico, al, mondo, ma, quello, che, ho, visto, nei, tiri, nei, passaggi, o, negli, interventi, della, cpu, pura, fantascienza, non, ero, rimasto, soddisfatt

In [0]:
lowered_text = reviews["body"][0:10].apply(lambda x: [item.lower() for item in x])
lowered_text.head(10)

0    [non, ho, mai, scritto, una, recensione, in, vita, mia, ma, per, questo, gioco, sento, proprio, il, bisogno, di, scriverci, qualcosa, parto, consigliando, a, tutti, di, non, acquistare, questo, gioco, perch, buttate, i, vostri, i, soldi, sono, arrivato, a, questa, conclusione, semplicemente, perch, che, tu, sia, bravo, o, meno, non, importa, e, il, gioco, che, decide, per, te, tra, 8, 9, gol, a, partita, o, errori, assurdi, fatti, dalla, cpu, il, tuo, giocare, passa, in, secondo, piano, ormai, credo, che, la, ea, preferisca, arricchirsi, con, vendite, di, giochi, e, fifa, points, senza, pensare, minimamente, al, gamplay, o, a, rendere, il, prodotto, ...]                                             
1    [uno, dei, peggiori, se, non, il, peggior, gioco, fifa, degli, ultimi, anni, dovrebbe, essere, il, gioco, di, calcio, pi, realistico, al, mondo, ma, quello, che, ho, visto, nei, tiri, nei, passaggi, o, negli, interventi, della, cpu, pura, fantascienza, non, ero, rimasto, soddisfatt

In [0]:
stop = stopwords.words("italian")

stopword_removed_text = lowered_text.apply(lambda x: [item for item in x if item not in stop])
stopword_removed_text

0    [mai, scritto, recensione, vita, gioco, sento, proprio, bisogno, scriverci, qualcosa, parto, consigliando, acquistare, gioco, perch, buttate, soldi, arrivato, conclusione, semplicemente, perch, bravo, meno, importa, gioco, decide, te, 8, 9, gol, partita, errori, assurdi, fatti, cpu, giocare, passa, secondo, piano, ormai, credo, ea, preferisca, arricchirsi, vendite, giochi, fifa, points, senza, pensare, minimamente, gamplay, rendere, prodotto, giocabile, gioco, fifa, ormai, 7, 8, anni, credo, peggiore, molto, deluso, detto, auguri, tutte, persone, compreranno, gioco, divertirsi]                                                                                                                                                                                                                                                            
1    [peggiori, peggior, gioco, fifa, ultimi, anni, dovrebbe, essere, gioco, calcio, pi, realistico, mondo, visto, tiri, passaggi, interventi, cpu, pura, fant

In [0]:
preprocessed_text = stopword_removed_text.apply(lambda x: [item for item in x if not item.isdigit()]) 
preprocessed_text.head(10)

0    [mai, scritto, recensione, vita, gioco, sento, proprio, bisogno, scriverci, qualcosa, parto, consigliando, acquistare, gioco, perch, buttate, soldi, arrivato, conclusione, semplicemente, perch, bravo, meno, importa, gioco, decide, te, gol, partita, errori, assurdi, fatti, cpu, giocare, passa, secondo, piano, ormai, credo, ea, preferisca, arricchirsi, vendite, giochi, fifa, points, senza, pensare, minimamente, gamplay, rendere, prodotto, giocabile, gioco, fifa, ormai, anni, credo, peggiore, molto, deluso, detto, auguri, tutte, persone, compreranno, gioco, divertirsi]                                                                                                                                                                                                                                                                                                        
1    [peggiori, peggior, gioco, fifa, ultimi, anni, dovrebbe, essere, gioco, calcio, pi, realistico, mondo, visto, tiri, passa